In [ ]:
!pip install py_entitymatching

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 14.9 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 
     |████████████████████████████████| 1.6 MB 52.8 MB/s 
     |████████████████████████████████| 661 kB 63.7 MB/s 
  Created wheel for py-entitymatching: filename=py_entitymatching-0.4.0-cp37-cp37m-linux_x86_64.whl size=2634202 sha256=9e55591a3ffbb0ae0360f62409c56d598992ae67f71c09b1a6c4e52ed11deb99
  Stored in directory: /root/.cache/pip/wheels/d3/ad/74/76750f1ddbf65ec97e582e64d7e3b0848ad21658d6a8ffbe56
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3703850 sha256=e3a63df1fd461ff6627bcd296280218194ae7c9dfdd6982a218a4765aa8e834a
  Stored in directory: /root/.cache/pip/wheels/7c/0d/a4/c0a779a4d196bed8ed41ca12be09a93e158e4fc974b59f9fd5
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linu

In [ ]:
# !pip install pandas_profiling==3.4.0

In [ ]:
# Import libraries
import py_entitymatching as em
import pandas as pd
import os, sys

In [ ]:
datasets_dir = em.get_install_path() + os.sep + 'datasets'

dblp_dataset_path = datasets_dir + os.sep + 'DBLP.csv'
acm_dataset_path = datasets_dir + os.sep + 'ACM.csv'

In [ ]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(dblp_dataset_path, key='id')
B = em.read_csv_metadata(acm_dataset_path, key='id')

In [ ]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2616
Number of tuples in B: 2294
Number of tuples in A X B (i.e the cartesian product): 6001104


In [ ]:
A.head()

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999.0
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996.0
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive Aggregate Functions,"Themistoklis Palpanas, Richard Sidle, Hamid Pirahesh, Roberta Cochrane",VLDB,2002.0
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Processing Algorithms in Object-Oriented Databases,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert Gruser",VLDB,1996.0
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spatial Output,"Erik G. Hoel, Hanan Samet",VLDB,1995.0


In [ ]:
B.head()

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA and databases","Athman Bouguettaya, Boualem Benatallah, Lily Hendra, James Beard, Kevin Smith, Mourad Quzzani",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Ludäscher, Richard Marciano, Yannis Papakonstantinou, Pave...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database system,"Alexander Brodsky, Victor E. Segal, Jia Chen, Paval A. Exarkhopoulo",International Conference on Management of Data,1999


In [ ]:
B[B['title']=="Book reviews"]

,id,title,authors,venue,year
512,601864,Book reviews,Karl Aberer,ACM SIGMOD Record,2002
599,604273,Book reviews,Karl Aberer,ACM SIGMOD Record,2001
619,637423,Book reviews,Karl Aberer,ACM SIGMOD Record,2002
668,507352,Book reviews,Karl Aberer,ACM SIGMOD Record,2002
745,565128,Book reviews,Karl Aberer,ACM SIGMOD Record,2002
2124,776993,Book reviews,Karl Aberer,ACM SIGMOD Record,2003
2129,959077,Book reviews,NaN,ACM SIGMOD Record,2003
2133,945740,Book reviews,NaN,ACM SIGMOD Record,2003


In [ ]:
em.get_key(A), em.get_key(B)

('id', 'id')

In [ ]:
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 500, 1)
# But for the demo, we will use the entire table A and B

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
# import pandas_profiling

In [ ]:
# pandas_profiling.ProfileReport(B)

### BLOCKING

#### Attribute Equivalence blocker

In [ ]:
## Attribute Equivalence blocker

ab = em.AttrEquivalenceBlocker()

In [ ]:
# Use block_tables to apply blocking over two input tables.
C1 = ab.block_tables(A, B,
                    l_block_attr='title', r_block_attr='title',
                    l_output_attrs=['title', 'authors', 'year'],
                    r_output_attrs=['title', 'authors', 'year'],
                    l_output_prefix='l_', r_output_prefix='r_')

In [ ]:
C1.head()

,_id,l_id,r_id,l_title,l_authors,l_year,r_title,r_authors,r_year
0,0,conf/vldb/PoosalaI96,673321,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",1996.0,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",1996
1,1,conf/vldb/GardarinGT96,673484,Cost-based Selection of Path Expression Processing Algorithms in Object-Oriented Databases,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert Gruser",1996.0,Cost-based Selection of Path Expression Processing Algorithms in Object-Oriented Databases,"Georges Gardarin, Jean-Robert Gruser, Zhao-Hui Tang",1996
2,2,conf/vldb/HoelS95,673135,Benchmarking Spatial Join Operations with Spatial Output,"Erik G. Hoel, Hanan Samet",1995.0,Benchmarking Spatial Join Operations with Spatial Output,"Erik G. Hoel, Hanan Samet",1995
3,3,conf/vldb/KemperK94,672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",1994.0,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",1994
4,4,journals/vldb/ChangG01,767145,Approximate query mapping: Accounting for translation closeness,"Hector Garcia-Molina, Kevin Chen-Chuan Chang",2001.0,Approximate query mapping: Accounting for translation closeness,"Kevin Chen-Chuan Chang, Héctor García-Molina",2001


In [ ]:
em.show_properties(C1)

id: 140372637443152
key: _id
fk_ltable: l_id
fk_rtable: r_id
ltable(obj.id): 140372641051920
rtable(obj.id): 140373170077840


In [ ]:
id(A)

140372641051920

Note that the metadata of C1 includes key, foreign key to the left and right tables (i.e A and B) and pointers to left and right tables.

## Handling Missing Values
If the input tuples have missing values in the blocking attribute, then they are ignored by default. This is because, including all possible tuple pairs with missing values can significantly increase the size of the candidate set. But if you want to include them, then you can set allow_missing paramater to be True.

In [ ]:
B[B['authors'].isna()]

,id,title,authors,venue,year
884,671838,"High-Performance and Scalability through Application Tier,In-Memory Data Management",NaN,Very Large Data Bases,2000
1146,673478,MineSet(tm): A System for High-End Data Mining and Visualization,NaN,Very Large Data Bases,1996
2128,959075,Database principles,NaN,ACM SIGMOD Record,2003
2129,959077,Book reviews,NaN,ACM SIGMOD Record,2003
2130,959082,Reports,NaN,ACM SIGMOD Record,2003
2131,945726,Special topic section on peer to peer data management,NaN,ACM SIGMOD Record,2003
2132,945738,Standards,NaN,ACM SIGMOD Record,2003
2133,945740,Book reviews,NaN,ACM SIGMOD Record,2003
2134,959063,Special section on sensor network technology and sensor data managment,NaN,ACM SIGMOD Record,2003
2136,758376,Author Index,NaN,Very Large Data Bases,2000


In [ ]:
C2 = ab.block_tables(A, B,
                    l_block_attr='title', r_block_attr='title',
                    l_output_attrs=['title', 'authors', 'year'],
                    r_output_attrs=['title', 'authors', 'year'],
                    l_output_prefix='l_', r_output_prefix='r_',
                     allow_missing=True)

In [ ]:
len(C1), len(C2)

(961, 961)

In [ ]:
# Instantiate Attr. Equivalence Blocker
ab = em.AttrEquivalenceBlocker()
# Use block_tables to apply blocking over two input tables.
C3 = ab.block_candset(C1, l_block_attr='year', r_block_attr='year')

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
len(C3)

854

In [ ]:
A.loc[[0]]

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999.0


In [ ]:
B.loc[[0]]

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999


#### Overlap Blocking

In [ ]:
# Instantiate overlap blocker object
ob = em.OverlapBlocker()

In [ ]:
# Specify the tokenization to be 'word' level and set overlap_size to be 3.
C4 = ob.block_tables(A, B, 'title', 'title', word_level=True, overlap_size=5,
                    l_output_attrs=['title', 'authors', 'year'],
                    r_output_attrs=['title', 'authors', 'year'],
                    show_progress=False)


/usr/local/lib/python3.7/dist-packages/py_stringsimjoin/utils/generic_helper.py:77: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  subset=[join_attr])


In [ ]:
C4.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,conf/sigmod/VossenW99,304586,The WASA2 Object-Oriented Workflow Management System,"Mathias Weske, Gottfried Vossen",1999.0,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",1999
1,1,conf/sigmod/CruzJ99,304587,A User-Centered Interface for Querying Distributed Multimedia Databases,"Kimberly M. James, Isabel F. Cruz",1999.0,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",1999
2,2,conf/sigmod/BouguettayaBH99,304589,"World Wide Database - Integrating the Web, CORBA, and Databases","Kevin Smith, Athman Bouguettaya, Mourad Ouzzani, James Beard, Lily Hendra, Boualem Benatallah",1999.0,"World Wide Database-integrating the Web, CORBA and databases","Athman Bouguettaya, Boualem Benatallah, Lily Hendra, James Beard, Kevin Smith, Mourad Quzzani",1999
3,3,conf/sigmod/NguyenS96,304589,Accessing Relational Databases from the World Wide Web,"Tam Nguyen, V. Srinivasan",1996.0,"World Wide Database-integrating the Web, CORBA and databases","Athman Bouguettaya, Boualem Benatallah, Lily Hendra, James Beard, Kevin Smith, Mourad Quzzani",1999
4,4,conf/sigmod/DattaDTVSR02,304589,Proxy-based acceleration of dynamically generated content on the world wide web: an approach and...,"Suresha, Kaushik Dutta, Krithi Ramamritham, Anindya Datta, Helen M. Thomas, Debra E. VanderMeer",2002.0,"World Wide Database-integrating the Web, CORBA and databases","Athman Bouguettaya, Boualem Benatallah, Lily Hendra, James Beard, Kevin Smith, Mourad Quzzani",1999


In [ ]:
len(C4)

3487

In [ ]:
# Specify the tokenization to be 'word' level and set overlap_size to be 3.
C5 = ob.block_tables(A, B, 'title', 'title', word_level=False, q_val=3, overlap_size=3,
                    l_output_attrs=['title', 'authors', 'year'],
                    r_output_attrs=['title', 'authors', 'year'],
                    show_progress=False)

/usr/local/lib/python3.7/dist-packages/py_stringsimjoin/utils/generic_helper.py:77: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  subset=[join_attr])


In [ ]:
len(C5)

3997983

In [ ]:
ob.stop_words

['a',
 'an',
 'and',
 'are',
 'as',
 'at',
 'be',
 'by',
 'for',
 'from',
 'has',
 'he',
 'in',
 'is',
 'it',
 'its',
 'on',
 'that',
 'the',
 'to',
 'was',
 'were',
 'will',
 'with']

#### Black Box Blocking

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 5.5 MB 62.9 MB/s 
     |████████████████████████████████| 1.3 MB 57.4 MB/s 
     |████████████████████████████████| 163 kB 59.2 MB/s 
     |████████████████████████████████| 7.6 MB 58.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e08a4ae30c9520bbeb9e7133bd51dee67d603834c8641bfa80a061b9686c4918
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# embeddings = model.encode(sentences)
# print(embeddings)

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
sentence1 = "I am a boy."
sentence2 = "I am a father."

In [ ]:
s1 = model.encode(sentence1)
s2 = model.encode(sentence2)

In [ ]:
s1.shape

(384,)

In [ ]:
import scipy

distances = scipy.spatial.distance.cdist([s1], [s2], "cosine")

In [ ]:
distances

In [ ]:
distances[0][0]

0.3528735959209154

In [ ]:
def similarity_check_title(x,y):
  x_title = x['title']
  y_title = y['title']

  x_title_embedding = model.encode(x_title)
  y_title_embedding = model.encode(y_title)

  distances = scipy.spatial.distance.cdist([x_title_embedding], [y_title_embedding], "cosine")

  if distances[0][0]<0.75:
    return True
  else:
    return False

In [ ]:
bb = em.BlackBoxBlocker()

bb.set_black_box_function(similarity_check_title)

In [ ]:
# C6 = bb.block_tables(A,B, l_output_attrs=['title','year'],r_output_attrs =['title','year'])

In [ ]:
# C6.head()

#### Rule Based Blocking:

In [ ]:
block_f = em.get_features_for_blocking(A1, B1, validate_inferred_attr_types=True)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,id,id,short string (1 word),numeric,Not Applicable: Types do not match
1,title,title,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,authors,authors,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
3,venue,venue,short string (1 word to 5 words),medium string (5 words to 10 words),Not Applicable: Types do not match
4,year,year,numeric,numeric,Exact Match; Absolute Norm


Do you want to proceed? (y/n):y


In [ ]:
em._block_c['corres']

[('id', 'id'),
 ('title', 'title'),
 ('authors', 'authors'),
 ('venue', 'venue'),
 ('year', 'year')]

In [ ]:
block_f.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x7f74a6097ef0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,title_title_cos_dlm_dc0_dlm_dc0,title,title,dlm_dc0,dlm_dc0,cosine,<function title_title_cos_dlm_dc0_dlm_dc0 at 0x7f74a6097dd0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,title_title_mel,title,title,None,None,monge_elkan,<function title_title_mel at 0x7f74a6097b00>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,title_title_lev_dist,title,title,None,None,lev_dist,<function title_title_lev_dist at 0x7f74a60ad0e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,title_title_lev_sim,title,title,None,None,lev_sim,<function title_title_lev_sim at 0x7f74a60ad200>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [ ]:
rb = em.RuleBasedBlocker()
# Add rule : block tuples if name_name_lev(ltuple, rtuple) < 0.4
rb.add_rule(['title_title_lev_dist(ltuple, rtuple) < 0.5'], block_f)

'_rule_0'

In [ ]:
C = rb.block_tables(A1, B1, l_output_attrs=['title', 'authors', 'year'],
                    r_output_attrs=['title', 'authors', 'year'], show_progress=False)


In [ ]:
C.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/sigmod/Mackay99,507360,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999.0,The XML typechecking problem,Dan Suciu,2002
1,1,journals/sigmod/Mackay99,271089,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999.0,Research issues in federated database systems: report of EFDBS '97 workshop,"S. Conrad, B. Eaglestone, W. Hasselbring, M. Roantree, M. Schöhoff, M. Strässler, M. Vermeer, F....",1997
2,2,journals/sigmod/Mackay99,672017,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999.0,ICICLES: Self-Tuning Samples for Approximate Query Answering,"Venkatesh Ganti, Mong-Li Lee, Raghu Ramakrishnan",2000
3,3,journals/sigmod/Mackay99,671027,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999.0,Scalable Techniques for Mining Causal Structures,"Craig Silverstein, Sergey Brin, Rajeev Motwani, Jeffrey D. Ullman",1998
4,4,journals/sigmod/Mackay99,767095,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999.0,Searching in metric spaces by spatial approximation,Gonzalo Navarro,2002


#### Features

In [ ]:
atypes1 = em.get_attr_types(A1)
atypes2 = em.get_attr_types(B1)

In [ ]:
print("A",atypes1.keys())
print("B",atypes2.keys())

A dict_keys(['id', 'title', 'authors', 'venue', 'year', '_table'])
B dict_keys(['id', 'title', 'authors', 'venue', 'year', '_table'])


In [ ]:
atypes1['id'],atypes1['authors'],atypes1['venue'],atypes1['year']

('str_eq_1w', 'str_bt_5w_10w', 'str_bt_1w_5w', 'numeric')

For example, str_bt_1w_5w means the average number of tokens in that column is greater than one word but less than 5 words.

In [ ]:
atypes2['id'],atypes2['authors'],atypes2['venue'],atypes2['year']

('numeric', 'str_bt_5w_10w', 'str_bt_5w_10w', 'numeric')

In [ ]:
block_c = em.get_attr_corres(A1, B1)

In [ ]:
block_c.keys()

dict_keys(['corres', 'ltable', 'rtable'])

In [ ]:
id(A1), id(block_c['ltable']), id(B1), id(block_c['rtable'])

(140139432335952, 140139432335952, 140139261638800, 140139261638800)

In [ ]:
block_c['corres']

[('id', 'id'),
 ('title', 'title'),
 ('authors', 'authors'),
 ('venue', 'venue'),
 ('year', 'year')]

In [ ]:
tok = em.get_tokenizers_for_blocking()

tok

{'qgm_2': <function py_entitymatching.feature.tokenizers._make_tok_qgram.<locals>.tok_qgram(s)>,
 'qgm_3': <function py_entitymatching.feature.tokenizers._make_tok_qgram.<locals>.tok_qgram(s)>,
 'wspace': <function py_entitymatching.feature.tokenizers.tok_wspace(input_string)>,
 'alphabetic': <function py_entitymatching.feature.tokenizers.tok_alphabetic(input_string)>,
 'alphanumeric': <function py_entitymatching.feature.tokenizers.tok_alphanumeric(input_string)>,
 'dlm_dc0': <function py_entitymatching.feature.tokenizers._make_tok_delim.<locals>.tok_delim(s)>}

In [ ]:
sim = em.get_sim_funs_for_blocking()

sim

{'affine': <function py_entitymatching.feature.simfunctions.affine(s1, s2)>,
 'hamming_dist': <function py_entitymatching.feature.simfunctions.hamming_dist(s1, s2)>,
 'hamming_sim': <function py_entitymatching.feature.simfunctions.hamming_sim(s1, s2)>,
 'lev_dist': <function py_entitymatching.feature.simfunctions.lev_dist(s1, s2)>,
 'lev_sim': <function py_entitymatching.feature.simfunctions.lev_sim(s1, s2)>,
 'jaro': <function py_entitymatching.feature.simfunctions.jaro(s1, s2)>,
 'jaro_winkler': <function py_entitymatching.feature.simfunctions.jaro_winkler(s1, s2)>,
 'needleman_wunsch': <function py_entitymatching.feature.simfunctions.needleman_wunsch(s1, s2)>,
 'smith_waterman': <function py_entitymatching.feature.simfunctions.smith_waterman(s1, s2)>,
 'overlap_coeff': <function py_entitymatching.feature.simfunctions.overlap_coeff(arr1, arr2)>,
 'jaccard': <function py_entitymatching.feature.simfunctions.jaccard(arr1, arr2)>,
 'dice': <function py_entitymatching.feature.simfunctions

In [ ]:
feature_table = em.get_features(A1, B1, atypes1, atypes2, block_c, tok, sim)

In [ ]:
feature_table.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x7f74a6035710>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,title_title_cos_dlm_dc0_dlm_dc0,title,title,dlm_dc0,dlm_dc0,cosine,<function title_title_cos_dlm_dc0_dlm_dc0 at 0x7f74a60353b0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,title_title_mel,title,title,None,None,monge_elkan,<function title_title_mel at 0x7f74a6035200>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,title_title_lev_dist,title,title,None,None,lev_dist,<function title_title_lev_dist at 0x7f74a6097830>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,title_title_lev_sim,title,title,None,None,lev_sim,<function title_title_lev_sim at 0x7f74a6097710>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [ ]:
feature_table.tail()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
9,authors_authors_lev_sim,authors,authors,None,None,lev_sim,<function authors_authors_lev_sim at 0x7f74b51975f0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
10,year_year_exm,year,year,None,None,exact_match,<function year_year_exm at 0x7f74b5197440>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
11,year_year_anm,year,year,None,None,abs_norm,<function year_year_anm at 0x7f74b51979e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
12,year_year_lev_dist,year,year,None,None,lev_dist,<function year_year_lev_dist at 0x7f74a60407a0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
13,year_year_lev_sim,year,year,None,None,lev_sim,<function year_year_lev_sim at 0x7f74a60403b0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


### Adding and removing features

For deleting we can delete row from df using delete.

1. Blackbox function and add it to feature table

To create and add a blackbox function as a feature, first you must define it. Specifically, the function must take in two tuples as input and return a numeric value.

In [ ]:
def year_diff(ltuple, rtuple):
  return ltuple['year']-rtuple['year']

In [ ]:
status = em.add_blackbox_feature(feature_table, 'release_difference', year_diff)

In [ ]:
feature_table.tail()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
10,year_year_exm,year,year,None,None,exact_match,<function year_year_exm at 0x7f74b5197440>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
11,year_year_anm,year,year,None,None,abs_norm,<function year_year_anm at 0x7f74b51979e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
12,year_year_lev_dist,year,year,None,None,lev_dist,<function year_year_lev_dist at 0x7f74a60407a0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
13,year_year_lev_sim,year,year,None,None,lev_sim,<function year_year_lev_sim at 0x7f74a60403b0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
14,release_difference,None,None,None,None,None,<function year_diff at 0x7f74b5aa3170>,None,False


2. Adding a Feature Declatatively

In [ ]:
r = em.get_feature_fn('jaccard(qgm_3(ltuple["title"]), qgm_3(rtuple["tile"]))', tok, sim)
em.add_feature(feature_table, 'title_title_jac_qgm3_qgm3', r)

True

In [ ]:
feature_table.feature_name

0             title_title_jac_qgm_3_qgm_3
1         title_title_cos_dlm_dc0_dlm_dc0
2                         title_title_mel
3                    title_title_lev_dist
4                     title_title_lev_sim
5         authors_authors_jac_qgm_3_qgm_3
6     authors_authors_cos_dlm_dc0_dlm_dc0
7                     authors_authors_mel
8                authors_authors_lev_dist
9                 authors_authors_lev_sim
10                          year_year_exm
11                          year_year_anm
12                     year_year_lev_dist
13                      year_year_lev_sim
14                     release_difference
15              title_title_jac_qgm3_qgm3
Name: feature_name, dtype: object

In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install python-Levenshtein==0.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 2.5 MB/s 
Reason for being yanked: Insecure, upgrade to 0.12.1
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp37-cp37m-linux_x86_64.whl size=145906 sha256=83f8147c3b3b68eee04bf6fa3ecb5d9f8ba743ce1f505c414a0b44cd6cf53c65
  Stored in directory: /root/.cache/pip/wheels/f0/9b/13/49c281164c37be18343230d3cd0fca29efb23a493351db0009
Successfully built python-Levenshtein


In [ ]:
import fuzzywuzzy.StringMatcher as fz

In [ ]:
fz.ratio('xyz', 'ayz')

0.6666666666666666

In [ ]:
sim['fz_ratio'] = fz.ratio

In [ ]:
sim

{'affine': <function py_entitymatching.feature.simfunctions.affine(s1, s2)>,
 'hamming_dist': <function py_entitymatching.feature.simfunctions.hamming_dist(s1, s2)>,
 'hamming_sim': <function py_entitymatching.feature.simfunctions.hamming_sim(s1, s2)>,
 'lev_dist': <function py_entitymatching.feature.simfunctions.lev_dist(s1, s2)>,
 'lev_sim': <function py_entitymatching.feature.simfunctions.lev_sim(s1, s2)>,
 'jaro': <function py_entitymatching.feature.simfunctions.jaro(s1, s2)>,
 'jaro_winkler': <function py_entitymatching.feature.simfunctions.jaro_winkler(s1, s2)>,
 'needleman_wunsch': <function py_entitymatching.feature.simfunctions.needleman_wunsch(s1, s2)>,
 'smith_waterman': <function py_entitymatching.feature.simfunctions.smith_waterman(s1, s2)>,
 'overlap_coeff': <function py_entitymatching.feature.simfunctions.overlap_coeff(arr1, arr2)>,
 'jaccard': <function py_entitymatching.feature.simfunctions.jaccard(arr1, arr2)>,
 'dice': <function py_entitymatching.feature.simfunctions

In [ ]:
feature_string = """fz_ratio((ltuple['title'] + ' ' + ltuple['authors']).lower(),
                            (rtuple['title'] + ' ' + rtuple['authors']).lower())"""
feature = em.get_feature_fn(feature_string, sim, tok)

# Add feature to F
em.add_feature(feature_table, 'fzratio_title_author', feature)

True

In [ ]:
feature_table.feature_name

0             title_title_jac_qgm_3_qgm_3
1         title_title_cos_dlm_dc0_dlm_dc0
2                         title_title_mel
3                    title_title_lev_dist
4                     title_title_lev_sim
5         authors_authors_jac_qgm_3_qgm_3
6     authors_authors_cos_dlm_dc0_dlm_dc0
7                     authors_authors_mel
8                authors_authors_lev_dist
9                 authors_authors_lev_sim
10                          year_year_exm
11                          year_year_anm
12                     year_year_lev_dist
13                      year_year_lev_sim
14                     release_difference
15              title_title_jac_qgm3_qgm3
16                   fzratio_title_author
Name: feature_name, dtype: object

### Sampling and Labeling

In [ ]:
C1.head()

,_id,l_id,r_id,l_title,l_authors,l_year,r_title,r_authors,r_year
0,0,conf/vldb/PoosalaI96,673321,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",1996.0,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",1996
1,1,conf/vldb/GardarinGT96,673484,Cost-based Selection of Path Expression Processing Algorithms in Object-Oriented Databases,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert Gruser",1996.0,Cost-based Selection of Path Expression Processing Algorithms in Object-Oriented Databases,"Georges Gardarin, Jean-Robert Gruser, Zhao-Hui Tang",1996
2,2,conf/vldb/HoelS95,673135,Benchmarking Spatial Join Operations with Spatial Output,"Erik G. Hoel, Hanan Samet",1995.0,Benchmarking Spatial Join Operations with Spatial Output,"Erik G. Hoel, Hanan Samet",1995
3,3,conf/vldb/KemperK94,672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",1994.0,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",1994
4,4,journals/vldb/ChangG01,767145,Approximate query mapping: Accounting for translation closeness,"Hector Garcia-Molina, Kevin Chen-Chuan Chang",2001.0,Approximate query mapping: Accounting for translation closeness,"Kevin Chen-Chuan Chang, Héctor García-Molina",2001


In [ ]:
S = em.sample_table(C1, 60)

In [ ]:
len(C1),len(S)

(961, 60)

In [ ]:
!pip install PyQt5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.4 MB 27.2 MB/s 
     |████████████████████████████████| 59.9 MB 13.1 MB/s 
     |████████████████████████████████| 344 kB 59.7 MB/s 


In [ ]:
G = em.label_table(S, label_column_name='gold_labels')

In [ ]:
path = "/content/drive/MyDrive/R.csv"

G = em.read_csv_metadata(path, key='_id',
                         fk_ltable='ltable_id', fk_rtable='rtable_id',
                         ltable=A, rtable=B)

In [ ]:
G.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year,gold_labels
0,7,conf/vldb/Bar-YossefBCFW00,671873,Approximating Aggregate Queries about Web Pages via Random Walks,"Jittat Fakcharoenphol, Steve Chien, Ziv Bar-Yossef, Alexander Berg, Dror Weitz",2000.0,Approximating Aggregate Queries about Web Pages via Random Walks,"Ziv Bar-Yossef, Alexander Berg, Steve Chien, Jittat Fakcharoenphol, Dror Weitz",2000,1
1,24,conf/sigmod/TatarinovIHW01,375720,Updating XML,"Daniel S. Weld, Zachary G. Ives, Igor Tatarinov, Alon Y. Halevy",2001.0,Updating XML,"Igor Tatarinov, Zachary G. Ives, Alon Y. Halevy, Daniel S. Weld",2001,1
2,34,conf/vldb/MoserKK95,673149,L/MRP: A Buffer Management Strategy for Interactive Continuous Data Flows in a Multimedia DBMS,"Wolfgang Klas, Frank Moser, Achim Kraiss",1995.0,L/MRP: A Buffer Management Strategy for Interactive Continuous Data Flows in a Multimedia DBMS,"Frank Moser, Achim Kraiss, Wolfgang Klas",1995,1
3,43,conf/vldb/ShaferA97,671018,Parallel Algorithms for High-dimensional Similarity Joins for Data Mining Applications,"Rakesh Agrawal, John C. Shafer",1997.0,Parallel Algorithms for High-dimensional Similarity Joins for Data Mining Applications,"John C. Shafer, Rakesh Agrawal",1997,1
4,56,conf/vldb/CatozziR95,673297,OS Support for VLDBs: Unix Enhancements for the Teradata Data Base,"Sorana Rabinovici, John Catozzi",1995.0,OS Support for VLDBs: Unix Enhancements for the Teradata Data Base,"John Catozzi, Sorana Rabinovici",1995,1


In [ ]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

#### Creating the Features Manually


In [ ]:
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()

In [ ]:
match_t

{'qgm_2': <function py_entitymatching.feature.tokenizers._make_tok_qgram.<locals>.tok_qgram(s)>,
 'qgm_3': <function py_entitymatching.feature.tokenizers._make_tok_qgram.<locals>.tok_qgram(s)>,
 'wspace': <function py_entitymatching.feature.tokenizers.tok_wspace(input_string)>,
 'alphabetic': <function py_entitymatching.feature.tokenizers.tok_alphabetic(input_string)>,
 'alphanumeric': <function py_entitymatching.feature.tokenizers.tok_alphanumeric(input_string)>,
 'dlm_dc0': <function py_entitymatching.feature.tokenizers._make_tok_delim.<locals>.tok_delim(s)>}

In [ ]:
match_s

{'affine': <function py_entitymatching.feature.simfunctions.affine(s1, s2)>,
 'hamming_dist': <function py_entitymatching.feature.simfunctions.hamming_dist(s1, s2)>,
 'hamming_sim': <function py_entitymatching.feature.simfunctions.hamming_sim(s1, s2)>,
 'lev_dist': <function py_entitymatching.feature.simfunctions.lev_dist(s1, s2)>,
 'lev_sim': <function py_entitymatching.feature.simfunctions.lev_sim(s1, s2)>,
 'jaro': <function py_entitymatching.feature.simfunctions.jaro(s1, s2)>,
 'jaro_winkler': <function py_entitymatching.feature.simfunctions.jaro_winkler(s1, s2)>,
 'needleman_wunsch': <function py_entitymatching.feature.simfunctions.needleman_wunsch(s1, s2)>,
 'smith_waterman': <function py_entitymatching.feature.simfunctions.smith_waterman(s1, s2)>,
 'overlap_coeff': <function py_entitymatching.feature.simfunctions.overlap_coeff(arr1, arr2)>,
 'jaccard': <function py_entitymatching.feature.simfunctions.jaccard(arr1, arr2)>,
 'dice': <function py_entitymatching.feature.simfunctions

In [ ]:
atypes1 = em.get_attr_types(A) # don't need, if atypes1 exists from blocking step
atypes2 = em.get_attr_types(B) # don't need, if atypes2 exists from blocking step
match_c = em.get_attr_corres(A, B)
match_f = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)

In [ ]:
match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x7f17061334d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,title_title_cos_dlm_dc0_dlm_dc0,title,title,dlm_dc0,dlm_dc0,cosine,<function title_title_cos_dlm_dc0_dlm_dc0 at 0x7f17061330e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,title_title_mel,title,title,None,None,monge_elkan,<function title_title_mel at 0x7f1706133680>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,title_title_lev_dist,title,title,None,None,lev_dist,<function title_title_lev_dist at 0x7f17061337a0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,title_title_lev_sim,title,title,None,None,lev_sim,<function title_title_lev_sim at 0x7f17061338c0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,authors_authors_jac_qgm_3_qgm_3,authors,authors,qgm_3,qgm_3,jaccard,<function authors_authors_jac_qgm_3_qgm_3 at 0x7f17061339e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,authors_authors_cos_dlm_dc0_dlm_dc0,authors,authors,dlm_dc0,dlm_dc0,cosine,<function authors_authors_cos_dlm_dc0_dlm_dc0 at 0x7f1706133b00>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,authors_authors_mel,authors,authors,None,None,monge_elkan,<function authors_authors_mel at 0x7f1706133c20>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,authors_authors_lev_dist,authors,authors,None,None,lev_dist,<function authors_authors_lev_dist at 0x7f1706133d40>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,authors_authors_lev_sim,authors,authors,None,None,lev_sim,<function authors_authors_lev_sim at 0x7f1706133e60>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


### Creating the Features Automatically


In [ ]:
match_a = em.get_features_for_matching(A, B)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,id,id,short string (1 word),numeric,Not Applicable: Types do not match
1,title,title,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,authors,authors,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
3,venue,venue,short string (1 word to 5 words),medium string (5 words to 10 words),Not Applicable: Types do not match
4,year,year,numeric,numeric,Exact Match; Absolute Norm


Do you want to proceed? (y/n):y


In [ ]:
em._match_t

{'qgm_2': <function py_entitymatching.feature.tokenizers._make_tok_qgram.<locals>.tok_qgram(s)>,
 'qgm_3': <function py_entitymatching.feature.tokenizers._make_tok_qgram.<locals>.tok_qgram(s)>,
 'wspace': <function py_entitymatching.feature.tokenizers.tok_wspace(input_string)>,
 'alphabetic': <function py_entitymatching.feature.tokenizers.tok_alphabetic(input_string)>,
 'alphanumeric': <function py_entitymatching.feature.tokenizers.tok_alphanumeric(input_string)>,
 'dlm_dc0': <function py_entitymatching.feature.tokenizers._make_tok_delim.<locals>.tok_delim(s)>}

### Train and Test split

In [ ]:
train_test = em.split_train_test(G, train_proportion=0.8)
train, test = train_test['train'], train_test['test']
print(len(train),len(test))

48 12


In [ ]:
train.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year,gold_labels
13,206,conf/vldb/CowleyP00,671837,Temporal Integrity Constraints with Indeterminacy,"Wes Cowley, Dimitris Plexousakis",2000.0,Temporal Integrity Constraints with Indeterminacy,"Wes Cowley, Dimitris Plexousakis",2000,1
41,643,conf/vldb/FerrandinaMZ94,672991,Implementing Lazy Database Updates for an Object Database System,"Fabrizio Ferrandina, Thorsten Meyer, Roberto Zicari",1994.0,Implementing Lazy Database Updates for an Object Database System,"Fabrizio Ferrandina, Thorsten Meyer, Roberto Zicari",1994,1
9,147,journals/sigmod/Snodgrass98a,604262,Reminiscences on Influential Papers,Richard T. Snodgrass,1998.0,Reminiscences on Influential Papers,Kenneth A. Ross,2001,1
25,351,conf/sigmod/BarbosaMKL02,564769,ToXgene: a template-based data generator for XML,"Alberto O. Mendelzon, John Keenleyside, Denilson Barbosa, Kelly A. Lyons",2002.0,ToXgene: a template-based data generator for XML,"Denilson Barbosa, Alberto Mendelzon, John Keenleyside, Kelly Lyons",2002,1
31,447,conf/vldb/JiangSLB00,672021,The BT-tree: A Branched and Temporal Access Method,"David B. Lomet, Betty Salzberg, Linan Jiang, Manuel Barrena Garca",2000.0,The BT-tree: A Branched and Temporal Access Method,"Linan Jiang, Betty Salzberg, David B. Lomet, Manuel Barrena García",2000,1


In [ ]:
match_a.feature_name

0             title_title_jac_qgm_3_qgm_3
1         title_title_cos_dlm_dc0_dlm_dc0
2                         title_title_mel
3                    title_title_lev_dist
4                     title_title_lev_sim
5         authors_authors_jac_qgm_3_qgm_3
6     authors_authors_cos_dlm_dc0_dlm_dc0
7                     authors_authors_mel
8                authors_authors_lev_dist
9                 authors_authors_lev_sim
10                          year_year_exm
11                          year_year_anm
12                     year_year_lev_dist
13                      year_year_lev_sim
Name: feature_name, dtype: object

In [ ]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(train,
                            feature_table=match_a,
                            attrs_after='gold_labels',
                            show_progress=False)

In [ ]:
H.head()

,_id,ltable_id,rtable_id,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold_labels
13,206,conf/vldb/CowleyP00,671837,1.0,1.0,1.0,0.0,1.0,1.000000,1.000000,1.000000,0.0,1.000000,1.0,1.000000,2.0,0.666667,1
41,643,conf/vldb/FerrandinaMZ94,672991,1.0,1.0,1.0,0.0,1.0,1.000000,1.000000,1.000000,0.0,1.000000,1.0,1.000000,2.0,0.666667,1
9,147,journals/sigmod/Snodgrass98a,604262,1.0,1.0,1.0,0.0,1.0,0.054054,0.000000,0.561111,14.0,0.300000,0.0,0.998501,6.0,0.000000,1
25,351,conf/sigmod/BarbosaMKL02,564769,1.0,1.0,1.0,0.0,1.0,0.772152,0.894427,0.800325,42.0,0.416667,1.0,1.000000,2.0,0.666667,1
31,447,conf/vldb/JiangSLB00,672021,1.0,1.0,1.0,0.0,1.0,0.821918,0.900000,0.892385,25.0,0.621212,1.0,1.000000,2.0,0.666667,1


In [ ]:
H.loc[18]

_id                                                        266
ltable_id                              conf/vldb/MiningGroup96
rtable_id                                               673478
title_title_jac_qgm_3_qgm_3                                1.0
title_title_cos_dlm_dc0_dlm_dc0                            1.0
title_title_mel                                            1.0
title_title_lev_dist                                       0.0
title_title_lev_sim                                        1.0
authors_authors_jac_qgm_3_qgm_3                            NaN
authors_authors_cos_dlm_dc0_dlm_dc0                        NaN
authors_authors_mel                                        NaN
authors_authors_lev_dist                                   NaN
authors_authors_lev_sim                                    NaN
year_year_exm                                              1.0
year_year_anm                                              1.0
year_year_lev_dist                                     

In [ ]:
H.loc[46]

_id                                                   728
ltable_id                              conf/vldb/StohrR01
rtable_id                                          672208
title_title_jac_qgm_3_qgm_3                           1.0
title_title_cos_dlm_dc0_dlm_dc0                       1.0
title_title_mel                                       1.0
title_title_lev_dist                                  0.0
title_title_lev_sim                                   1.0
authors_authors_jac_qgm_3_qgm_3                  0.827586
authors_authors_cos_dlm_dc0_dlm_dc0                  0.75
authors_authors_mel                                 0.992
authors_authors_lev_dist                              1.0
authors_authors_lev_sim                              0.96
year_year_exm                                         NaN
year_year_anm                                         NaN
year_year_lev_dist                                    NaN
year_year_lev_sim                                     NaN
gold_labels   

In [ ]:
H[H['authors_authors_jac_qgm_3_qgm_3'].isnull()].index.tolist()

[18]

In [ ]:
H[H['year_year_lev_sim'].isnull()].index.tolist()

[46]

In [ ]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.notnull(H))

True

#### Handling missing values

All ways of pandas can be used as result is dataframe or use impute_table.


In [ ]:
import numpy as np
# Impute feature vectors with the mean of the column values.
H = em.impute_table(H,
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
                missing_val=np.nan,
                strategy='median')

/usr/local/lib/python3.7/dist-packages/py_entitymatching/matcher/matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


#### FIt and Transform



In [ ]:
dt.fit(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], target_attr='gold_labels')

In [ ]:
dt.predict(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], target_attr='predicted_labels', return_probs=True, probs_attr='proba', append=True,
inplace=True)

,_id,ltable_id,rtable_id,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold_labels,predicted_labels,proba
13,206,conf/vldb/CowleyP00,671837,1.0,1.0,1.0,0.0,1.0,1.000000,1.000000,1.000000,0.0,1.000000,1.0,1.000000,2.0,0.666667,1,1,1.0
41,643,conf/vldb/FerrandinaMZ94,672991,1.0,1.0,1.0,0.0,1.0,1.000000,1.000000,1.000000,0.0,1.000000,1.0,1.000000,2.0,0.666667,1,1,1.0
9,147,journals/sigmod/Snodgrass98a,604262,1.0,1.0,1.0,0.0,1.0,0.054054,0.000000,0.561111,14.0,0.300000,0.0,0.998501,6.0,0.000000,1,1,1.0
25,351,conf/sigmod/BarbosaMKL02,564769,1.0,1.0,1.0,0.0,1.0,0.772152,0.894427,0.800325,42.0,0.416667,1.0,1.000000,2.0,0.666667,1,1,1.0
31,447,conf/vldb/JiangSLB00,672021,1.0,1.0,1.0,0.0,1.0,0.821918,0.900000,0.892385,25.0,0.621212,1.0,1.000000,2.0,0.666667,1,1,1.0
54,876,conf/vldb/BalminPP00,672016,1.0,1.0,1.0,0.0,1.0,0.870968,1.000000,0.911111,30.0,0.500000,1.0,1.000000,2.0,0.666667,1,1,1.0
56,907,conf/vldb/BaumannFRW97,673652,1.0,1.0,1.0,0.0,1.0,0.800000,0.750000,0.792929,33.0,0.450000,1.0,1.000000,2.0,0.666667,1,1,1.0
55,892,conf/vldb/BrownCC94,672985,1.0,1.0,1.0,0.0,1.0,0.725490,0.750000,0.754710,22.0,0.521739,1.0,1.000000,2.0,0.666667,1,1,1.0
52,862,conf/vldb/FegarasE01,672209,1.0,1.0,1.0,0.0,1.0,0.609756,0.500000,0.613441,27.0,0.129032,1.0,1.000000,2.0,0.666667,1,1,1.0
2,34,conf/vldb/MoserKK95,673149,1.0,1.0,1.0,0.0,1.0,0.750000,0.666667,0.764286,30.0,0.250000,1.0,1.000000,2.0,0.666667,1,1,1.0


#### We also have rule based matching: Similar to rule based blocking

### Selecting the best matcher using cross-validation


In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H,
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 199, in fit
    y = self._validate_targets(y)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 722, in _validate_targets
    % len(cls)
ValueError: The number of classes has to be greater than one; got 1 class

  warning

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,1.0,1.0,1.0
1,RF,1.0,1.0,1.0
2,SVM,NaN,NaN,NaN
3,LinReg,1.0,1.0,1.0
4,LogReg,NaN,NaN,NaN


In [ ]:
result['drill_down_cv_stats']['precision']


,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f1706989f50>,5,1.0,1.0,1.0,1.0,1.0,1.0
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f17069895d0>,5,1.0,1.0,1.0,1.0,1.0,1.0
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f17069896d0>,5,NaN,NaN,NaN,NaN,NaN,NaN
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f1706989210>,5,1.0,1.0,1.0,1.0,1.0,1.0
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f1706989e90>,5,NaN,NaN,NaN,NaN,NaN,NaN


### Combining multiple matcher

In [ ]:
dt.fit(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id','gold_labels'], target_attr='gold_labels') # H is training set containing feature vectors
dt.predict(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id','gold_labels'], append=True, inplace=True, target_attr='dt_predictions') # L is the test set for which we should get predictions.
rf.fit(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id','gold_labels'], target_attr='gold_labels')
rf.predict(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id','gold_labels'], append=True, inplace=True, target_attr='rf_predictions')

,_id,ltable_id,rtable_id,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,...,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold_labels,predicted_labels,proba,dt_predictions,rf_predictions
13,206,conf/vldb/CowleyP00,671837,1.0,1.0,1.0,0.0,1.0,1.000000,1.000000,...,1.000000,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
41,643,conf/vldb/FerrandinaMZ94,672991,1.0,1.0,1.0,0.0,1.0,1.000000,1.000000,...,1.000000,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
9,147,journals/sigmod/Snodgrass98a,604262,1.0,1.0,1.0,0.0,1.0,0.054054,0.000000,...,0.300000,0.0,0.998501,6.0,0.000000,1,1,1.0,1,1
25,351,conf/sigmod/BarbosaMKL02,564769,1.0,1.0,1.0,0.0,1.0,0.772152,0.894427,...,0.416667,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
31,447,conf/vldb/JiangSLB00,672021,1.0,1.0,1.0,0.0,1.0,0.821918,0.900000,...,0.621212,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
54,876,conf/vldb/BalminPP00,672016,1.0,1.0,1.0,0.0,1.0,0.870968,1.000000,...,0.500000,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
56,907,conf/vldb/BaumannFRW97,673652,1.0,1.0,1.0,0.0,1.0,0.800000,0.750000,...,0.450000,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
55,892,conf/vldb/BrownCC94,672985,1.0,1.0,1.0,0.0,1.0,0.725490,0.750000,...,0.521739,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
52,862,conf/vldb/FegarasE01,672209,1.0,1.0,1.0,0.0,1.0,0.609756,0.500000,...,0.129032,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
2,34,conf/vldb/MoserKK95,673149,1.0,1.0,1.0,0.0,1.0,0.750000,0.666667,...,0.250000,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1


### Trigger

In [ ]:
mt = em.MatchTrigger()

In [ ]:
# Add two rules to the rule-based matcher

# Since we removed all of the features comparing Title earlier, we want to now add a rule that compares Titles
mt.add_cond_rule(['title_title_lev_sim(ltuple, rtuple) > 0.7'], match_a)
# The rule has two predicates, one comparing the titles and the other looking for an exact match of the years
mt.add_cond_rule(['title_title_lev_sim(ltuple, rtuple) > 0.4', 'year_year_exm(ltuple, rtuple) == 1'], match_a)
mt.get_rule_names()

odict_keys(['_rule_0', '_rule_1'])

In [ ]:
# Rules can also be deleted from the rule-based matcher
mt.delete_rule('_rule_1')

True

Next, we need to add a condition status and an action to the trigger. Triggers apply the rules added to each tuple pair. If the result is the same value as the condition status, then the action will be carried out.

In [ ]:
# Since we are using the trigger to fix a problem related to false negatives, we want the condition to be
# True and the action to be 1. This way, the trigger will set a prediction to 1 when the rule returns True.

mt.add_cond_status(True)
mt.add_action(1)

True

In [ ]:
preds = mt.execute(input_table=H, label_column='predicted_labels', inplace=False)
preds.head()

,_id,ltable_id,rtable_id,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,...,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold_labels,predicted_labels,proba,dt_predictions,rf_predictions
13,206,conf/vldb/CowleyP00,671837,1.0,1.0,1.0,0.0,1.0,1.000000,1.000000,...,1.000000,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
41,643,conf/vldb/FerrandinaMZ94,672991,1.0,1.0,1.0,0.0,1.0,1.000000,1.000000,...,1.000000,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
9,147,journals/sigmod/Snodgrass98a,604262,1.0,1.0,1.0,0.0,1.0,0.054054,0.000000,...,0.300000,0.0,0.998501,6.0,0.000000,1,1,1.0,1,1
25,351,conf/sigmod/BarbosaMKL02,564769,1.0,1.0,1.0,0.0,1.0,0.772152,0.894427,...,0.416667,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1
31,447,conf/vldb/JiangSLB00,672021,1.0,1.0,1.0,0.0,1.0,0.821918,0.900000,...,0.621212,1.0,1.000000,2.0,0.666667,1,1,1.0,1,1


In [ ]:
preds[preds['gold_labels'] != preds['predicted_labels']]

,_id,ltable_id,rtable_id,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,...,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold_labels,predicted_labels,proba,dt_predictions,rf_predictions


In [ ]:
eval_result = em.eval_matches(H,"gold_labels","predicted_labels")

In [ ]:
em.print_eval_summary(eval_result)

Precision : 100.0% (48/48)
Recall : 100.0% (48/48)
F1 : 100.0%
False positives : 0 (out of 48 positive predictions)
False negatives : 0 (out of 0 negative predictions)
